In [36]:

#Objectifs:
#  Programmer sous R:
#    1) Une fonction pour la (TRAIN-TEST) validation répétée
#    2) Une fonction pour la validation croisée (K feuilles)
#    3) Une fonction pour la validation croisée imbriquée
#    4) Une fonction pour le Leave One Out (avec k = n)
# 
# deux critères à valider: moyenne des performances et écart-type des 

# construction de modèles (ou librairies de notre choix):
# library(rpart) = algos de décisions -> CART: classification and regression tree (classif ou regression)
# library(MASS) = algo de décisions -> LDA : linear decision analysis: (classif binaire)
# library(FNN) : K plus proche voisin (k-PPV)
# tree <- rpart(X10 ~ ., data= mesData, method= 'class')  ici ~ = en fonction de et . = toutes les autres variables
# tree <- rpart(X10 ~ ., data= mesData, method= 'class', control=rpart.control(minsplit=50), cp= val entre 0 et 1)  ici ~ = en fonction de et . = toutes les autres variables
# predict(tree, mesNouvellesDonnées, type='class')
# predict(tree, mesNouvellesDonnées, type='prob')
# JEUX de données madoc: Breiman: wave5000

# attente livrable: les fonctions, mini rapport explication fonction et comparaison des modèles(1,2,3,4) :robustesse sur les données de Breiman

# en sortie un vecteur mélanger de 1 à n: en entré un entier
# fonction sample déjà défini


# Question 1
#data = dataframe avec une colonne réel
#k = le nombre de répétition
#predictColumn = string = nom de la colonne X10
library(rpart)
validationRepeteeRPART = function (data, k) {
  errorRate = c();
  for (i in 1:k) {
    #on mélange les données
    randomData = data[sample(nrow(data)),]
    #les 20% premières lignes sont les données de tests
    testData = randomData[1: 1000,]
    #les 80% autres sont les données pour la construction de modèle
    modalData = randomData[1000:nrow(data),]
    #on calcule la précision (le taux d'erreurs)
    tree = rpart(modalData$X10~.,modalData, method="class")
    prediction = predict(tree, testData, type="class")
    print(prediction)
    #utiliser predict pour trouver le taux d'erreur entre data et predict puis utiliser table
    contingencyTable = table(testData$X10, prediction)
    
    #dans une matrice de confusion, les individus en diagonale sont les biens classés donc pour avoir le taux d'erreur on calcule la somme de tous les autres cases/la somme totale de toutes les cases
    sumError = 0
    total = 0
    for (i in 1:nrow(contingencyTable)) {
      for (j in 1:ncol(contingencyTable)) {
        if (i != j) {
          sumError = sumError + contingencyTable[i,j]
        }
        total = total + contingencyTable[i,j]
      }
    }
    errorRate = c(errorRate, c(sumError/total))
  }
  return(mean(errorRate))
}

library(MASS)

validationRepeteeLDA = function (data, k) {
  errorRate = c()
  for (i in 1:k) {
    #on mélange les données
    randomData = data[sample(nrow(data)),]
    #les 20% premières lignes sont les données de tests
    testData = randomData[1: 1000,]
    #les 80% autres sont les données pour la construction de modèle
    modalData = randomData[1000:nrow(data),]
    #on calcule la précision (le taux d'erreurs)
    tree = lda(modalData$X10~.,modalData, method="moment")
    prediction = predict(tree, testData, type="moment")
    #utiliser predict pour trouver le taux d'erreur entre data et predict puis utiliser table
    contingencyTable = table(testData$X10, prediction$class)
    #dans une matrice de confusion, les individus en diagonale sont les biens classés donc pour avoir le taux d'erreur on calcule la somme de tous les autres cases/la somme totale de toutes les cases
    sumError = 0
    total = 0
    for (i in 1:nrow(contingencyTable)) {
      for (j in 1:ncol(contingencyTable)) {
        if (i != j) {
          sumError = sumError + contingencyTable[i,j]
        }
        total = total + contingencyTable[i,j]
      }
    }
    errorRate = c(errorRate, c(sumError/total))
  }
  return(mean(errorRate))
}

#Exercice 2
#data = dataframe
#partitions = integer
validationCroiseeRPART = function(data, partitions, metric) {
  errorRate = c()
  #on mélange les données
  randomData = data[sample(nrow(data)),]
  #list
  setPartitions = split(randomData, 1:partitions)
  for (i in 1:(partitions)) {
    tempSetPartitions = setPartitions
    currentTestData = setPartitions[[i]]
    currentModalDataList = tempSetPartitions[-i]
    #fusion dataframe
    currentModalData = rbind.data.frame(currentModalDataList[[1]], currentModalDataList[[2]])
    
    for (j in 3:(partitions-1)) {
      currentModalData = rbind.data.frame(currentModalData, currentModalDataList[[j]])
    }
    
    numOfLinesTestCurrentModalData = as.integer(length(currentModalData)*0.2)
    numOflinesModalCurrentModalData = as.integer(length(currentModalData)*0.8)
    
    testData_step2 = currentModalData[1:numOfLinesTestCurrentModalData,]
    modalData_step2 = currentModalData[numOfLinesTestCurrentModalData+1:numOflinesModalCurrentModalData,]
    
    #on calcule la précision (le taux d'erreurs)
    tree = rpart(modalData_step2$X10~.,modalData_step2, method="class")
    prediction = predict(tree, testData_step2, type="class")
    #utiliser predict pour trouver le taux d'erreur entre data et predict puis utiliser table
    contingencyTable = table(testData_step2$X10, prediction)
    
    total = 0
    sumError = 0
    for (i in 1:nrow(contingencyTable)) {
      for (j in 1:ncol(contingencyTable)) {
        if (i != j) {
          sumError = sumError + contingencyTable[i,j]
        }
        total = total + contingencyTable[i,j]
      }
    }
    errorRate = c(errorRate, c(sumError/total))
    
  }
  
  meanPrecision = mean(errorRate)
  robustesse_calculate = 0
  
  for (i in 1:length(errorRate)) {
    robustesse_calculate = robustesse_calculate + (errorRate[i] - meanPrecision)*(errorRate[i] - meanPrecision)
  }
  #nombre de valeurs dans error rate est la même que le nombre de partitions
  robustesse = robustesse_calculate/length(errorRate)
  
  if (metric == 1) {
    return(robustesse)
  } else {
    return(mean(errorRate))
  }
  
}




validationCroiseeLDA = function(data, partitions, metric) {
  errorRate = c()
  #on mélange les données
  randomData = data[sample(nrow(data)),]
  #list
  setPartitions = split(randomData, 1:partitions)
  for (i in 1:(partitions)) {
    tempSetPartitions = setPartitions
    currentTestData = setPartitions[[i]]
    currentModalDataList = tempSetPartitions[-i]
    #fusion dataframe
    currentModalData = rbind.data.frame(currentModalDataList[[1]], currentModalDataList[[2]])
    
    for (j in 3:(partitions-1)) {
      currentModalData = rbind.data.frame(currentModalData, currentModalDataList[[j]])
    }
    
    numOfLinesTestCurrentModalData = as.integer(length(currentModalData)*0.2)
    numOflinesModalCurrentModalData = as.integer(length(currentModalData)*0.8)
    
    testData_step2 = currentModalData[1:numOfLinesTestCurrentModalData,]
    modalData_step2 = currentModalData[numOfLinesTestCurrentModalData+1:numOflinesModalCurrentModalData,]
    
    #on calcule la précision (le taux d'erreurs)
    tree = lda(X10~.,modalData_step2)
    prediction = predict(tree, testData_step2, type="class")
    #utiliser predict pour trouver le taux d'erreur entre data et predict puis utiliser table
    contingencyTable = table(testData_step2$X10, prediction$class)
    
    total = 0
    sumError = 0
    for (i in 1:nrow(contingencyTable)) {
      for (j in 1:ncol(contingencyTable)) {
        if (i != j) {
          sumError = sumError + contingencyTable[i,j]
        }
        total = total + contingencyTable[i,j]
      }
    }
    errorRate = c(errorRate, c(sumError/total))
    
  }
  
  meanPrecision = mean(errorRate)
  robustesse_calculate = 0
  
  for (i in 1:length(errorRate)) {
    robustesse_calculate = robustesse_calculate + (errorRate[i] - meanPrecision)*(errorRate[i] - meanPrecision)
  }
  #nombre de valeurs dans error rate est la même que le nombre de partitions
  robustesse = robustesse_calculate/length(errorRate)
  
  if (metric == 1) {
    return(robustesse)
  } else {
    return(mean(errorRate))
  }
  
}


#appel de fonction validationCroisee défini précédemment
#en paramètre k = le nombre de répétition
validationCroiseeRepetee = function (data, partitions, k) {
  robustesseMean = 0
  errorRateMean = 0
  
  for (i in 1:k) {
    robustesseMean = robustesseMean + validationCroiseeRPART(data, partitions, 1)
    errorRateMean = errorRateMean + validationCroiseeRPART(data, partitions, 0)
  }
  
  robustesseMean = robustesseMean/k
  errorRateMean = errorRateMean/k
  
  return(c(robustesseMean, errorRateMean))
  
}

#data = dataframe
#partitions = nombre de partition où on va split le dataset
validationCroiseeImpliquee = function(data, partitions) {
  
  errorRate = c()
  #on mélange les données
  randomData = data[sample(nrow(data)),]
  #list
  setPartitions = split(randomData, 1:partitions)
  listErrorRate = c()
  
  for (i in 1:partitions) {
    
    errorRate = c()
    
    for (i in 1:partitions-1) {
 
      tempSetPartitions = setPartitions
      currentTestData = setPartitions[[i]]
      currentModalDataList = tempSetPartitions[-i]
      #fusion dataframe
      currentModalData = rbind.data.frame(currentModalDataList[[1]], currentModalDataList[[2]])
      
      for (j in 3:(partitions-2)) {
        currentModalData = rbind.data.frame(currentModalData, currentModalDataList[[j]])
      }
      
      numOfLinesTestCurrentModalData = as.integer(length(currentModalData)*0.2)
      numOflinesModalCurrentModalData = as.integer(length(currentModalData)*0.8)
      
      testData_step2 = currentModalData[1:numOfLinesTestCurrentModalData,]
      modalData_step2 = currentModalData[numOfLinesTestCurrentModalData+1:numOflinesModalCurrentModalData,]
      
      #on calcule la précision (le taux d'erreurs)
      tree = lda(X10~.,modalData_step2)
      prediction = predict(tree, testData_step2, type="class")
      #utiliser predict pour trouver le taux d'erreur entre data et predict puis utiliser table
      contingencyTable = table(testData_step2$X10, prediction$class)
      
      total = 0
      sumError = 0
      for (i in 1:nrow(contingencyTable)) {
        for (j in 1:ncol(contingencyTable)) {
          if (i != j) {
            sumError = sumError + contingencyTable[i,j]
          }
          total = total + contingencyTable[i,j]
        }
      }
      errorRate = c(errorRate, c(sumError/total))
    }
    
    listErrorRate = c(c(listErrorRate), mean(errorRate))
    
  }
  
  return(listErrorRate)
  
}

#appel de la fonction validationCroisee
#en paramètre data: un dataframe
leaveOneOut = function(data) {
  numberOfrows = nrow(data)
  return(c(validationCroiseeRPART(data, numberOfrows, 0), validationCroiseeRPART(data, numberOfrows, 1)))
}


#appel de tous les fonctions définies précédemment

algorithmSelection = function(data, K) {
  
  validationCroiseeRPARTErrorRate = validationRepeteeLDA(data, K)
  validationCroiseeLDAErrorRate = validationRepeteeRPART(data, K)
  
  if (validationCroiseeRPARTErrorRate > validationCroiseeLDAErrorRate) {
    return("SOLUTION: LDA")
  } else {
    return("SOLUTION: RPART")
  }
  
  return("ERROR")
}




In [37]:
library(readr)
apprentissage <- read_csv(file = "C:/Users/Valentin/MiniProjetAFD/projetEntrepot/apprentissage.csv",col_names = FALSE)
apprentissage <- apprentissage[ ,- c(1,2,3,4,5,6,7,8,9,11)]


Parsed with column specification:
cols(
  .default = col_double(),
  X1 = col_character(),
  X6 = col_character(),
  X7 = col_integer(),
  X8 = col_character(),
  X10 = col_integer(),
  X11 = col_integer(),
  X21 = col_integer()
)
See spec(...) for full column specifications.
Warning message in rbind(names(probs), probs_f):
"number of columns of result is not a multiple of vector length (arg 1)"Warning message:
"5 parsing failures.
row # A tibble: 5 x 5 col     row col   expected   actual file                                            expected   <int> <chr> <chr>      <chr>  <chr>                                           actual 1  3279 X7    an integer M1     'C:/Users/Valentin/MiniProjetAFD/projetEntrepo~ file 2  3683 X7    an integer M2     'C:/Users/Valentin/MiniProjetAFD/projetEntrepo~ row 3  3823 X7    an integer M1     'C:/Users/Valentin/MiniProjetAFD/projetEntrepo~ col 4  3948 X7    an integer M1     'C:/Users/Valentin/MiniProjetAFD/projetEntrepo~ expected 5 11522 X7    an int

In [41]:
print(apprentissage)
validationRepeteeRPART(apprentissage, 10)
validationCroiseeRPART(apprentissage, 5, 0)


# A tibble: 11,566 x 31
      X10     X12     X13   X14    X15     X16   X17    X18   X19     X20   X21
    <int>   <dbl>   <dbl> <dbl>  <dbl>   <dbl> <dbl>  <dbl> <dbl>   <dbl> <int>
 1      0    0       0        0    0   19169       0 2.95e2   0       0       0
 2      0    0       0        0    0   31991       0 7.08e0   0      34.1     0
 3      0    0       0        0    0   29740       0 8.07e2   0       0       0
 4  17500 1935    2135     1192 1244     790     517 3.41e3 637   -  53.1     0
 5  19200  124       0      329    0       0       0 1.47e2   0       0       0
 6  76800  285       0    82691   68.4    51.8     0 1.69e3   0    1945       0
 7      0   10.5    37.3      0    0       0       0 1.02e1   0       0       0
 8      0   56.4     0        0    0       0     176 2.24e1   0       0       0
 9  17500    8.22    5.67     0    0       0       0 0        0       0       0
10 159600  103     528      386   90.9  1453       0 3.03e2  18.6     0       0
# ... with 11,55

[1] 0.6469

Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
"la taille de données n'est pas un multiple de la variable découpée"

[1] 0.9666667